# Question 1: Enron Analysis

In [1]:
import os
import glob,codecs
import csv
import math
import re
from email.parser import HeaderParser#To get the headers in the emails
relative_path='/Users/Sneha/Downloads/maildir'#giving relative path, please use your absolute path here
enron_files = [ef for ef in os.listdir(relative_path)]
#print(enron_files)
global count_mails
count_mails=0
sent_dict={}#dictionary for sent emails
recv_dict={}#dictionary for recieved emails
last_names=[]#to append the last names of the people to this list
G_Links = []#List to append to the exisiting url list

#To split the last name in the folder name
parser = HeaderParser()
for a in enron_files:
    last_n= a.split('-')
    last_names.append(last_n[0])

#Defining a function to check if the encountered is a folder or a file
#If folder then add to the existing path if not else command will get executed which will open and read the text files
def check_file_or_dir(f,case):
    global count_mails
    if os.path.isdir(f):
        #print ("i am a directory", f)
        dir_path=[p for p in os.listdir(f)]
        for w in dir_path:
            if w!='.DS_Store':
                new_files=f+'/'+w
                check_file_or_dir(new_files,case)
    else:
        if case==1:#To find the number of mails communicated in Dec 2011
            with codecs.open(f, "r",encoding='utf-8', errors='ignore') as fdata:#Encoding the data in a readable format
                b=fdata.read()
                h = parser.parsestr(b)
                headers= h.items()
                for z in headers:#Key value pairs which has key as the Date and value as Dec 2011
                    head,value=z
                    if head=='Date':
                        if value.find('Dec 2001')!=-1:
                            count_mails=count_mails+1#Counting the number of instances of the mails communicated in dec2011
        if case==2:#To find the number of mails sent and recieved by all the people in the enron dataset
            with codecs.open(f, "r",encoding='utf-8', errors='ignore') as fdata:
                b=fdata.read()
                h = parser.parsestr(b)
                headers= h.items()
                for z in headers:
                    head,value=z
                    if head=='From':
                        From=value
                        for l in last_names:
                            if l in From:
                                if l not in sent_dict:
                                    sent_dict[l]=1
                                else:
                                    c=sent_dict[l]
                                    sent_dict[l]=c+1
                            
                    if head=='To':
                        To=value
                        for l in last_names:
                                if l in To:
                                    if l not in recv_dict:
                                        recv_dict[l]=1
                                    else:
                                        c=recv_dict[l]
                                        recv_dict[l]=c+1
        if case==3:#To check the Urls in the body of the emails and adding them to the text file
            with codecs.open(f, "r",encoding='utf-8', errors='ignore') as fdata:
                b=fdata.read()
                #print (b)
                #h = parser.parsestr(b)
            #use re.findall to get all the links
                links= re.findall('(https?://[^\s]+)', b)#Regex to find URL in email body
            G_Links.extend(links)#
                        
for i in enron_files:#For executing all the three cases
    if i!='.DS_Store':
        folder_path=relative_path+'/'+i
        inside_folder=[f for f in os.listdir(folder_path)]
        #print (inside_folder)
        for j in inside_folder:
            files=folder_path+'/'+j
            check_file_or_dir(files,1)
        for j in inside_folder:
            files=folder_path+'/'+j
            check_file_or_dir(files,2)
        for j in inside_folder:
            files=folder_path+'/'+j
            check_file_or_dir(files,3)
print ('The total number of mails send in Dec 2001 were:', count_mails)#For the first case 

if os.path.isfile('/Users/Sneha/URL.txt'):#If the file already exists we delete and create a new file
    os.remove('/Users/Sneha/URL.txt')
j=set(G_Links)
for i in j:
    #print (i)
    file=open('URL.txt','a')
    file.write(i+'\n')

if os.path.isfile('/Users/Sneha/output_enron_file.csv'):#If the file already exists we delete and create a new file
    os.remove('/Users/Sneha/output_enron_file.csv')
for a in last_names:
    if a!='.DS_Store':
        if a in sent_dict:# counting the numer of mails sent by a particular employee
            sent_mail=sent_dict[a]
        else:
            sent_mail=0
        if a in recv_dict:# counting the numer of mails sent by a particular employee
            recv_mail=recv_dict[a]
        else:
            recv_mail=0
    #Writing the output to a csv file    
    with open("output_enron_file.csv",'a') as result:
        if a!='.DS_Store':
            wr=csv.writer(result, dialect='excel')
            row_enron=[a,sent_mail,recv_mail]
            wr.writerow(row_enron)
                    
   
            
    

The total number of mails send in Dec 2001 were: 11318
